In [2]:
# !pip install opencv-python

   ---------------------------------------- 0.0/38.8 MB ? eta -:--:--
   ---------------------------------------- 0.0/38.8 MB ? eta -:--:--
   ---------------------------------------- 0.0/38.8 MB 487.6 kB/s eta 0:01:20
   ---------------------------------------- 0.1/38.8 MB 558.5 kB/s eta 0:01:10
   ---------------------------------------- 0.1/38.8 MB 847.9 kB/s eta 0:00:46
   ---------------------------------------- 0.2/38.8 MB 953.7 kB/s eta 0:00:41
   ---------------------------------------- 0.3/38.8 MB 1.4 MB/s eta 0:00:29
   ---------------------------------------- 0.5/38.8 MB 1.4 MB/s eta 0:00:28
    --------------------------------------- 0.6/38.8 MB 1.5 MB/s eta 0:00:26
    --------------------------------------- 0.7/38.8 MB 1.6 MB/s eta 0:00:24
    --------------------------------------- 0.7/38.8 MB 1.7 MB/s eta 0:00:23
    --------------------------------------- 0.8/38.8 MB 1.6 MB/s eta 0:00:24
    --------------------------------------- 0.9/38.8 MB 1.7 MB/s eta 0:00:22
   - 

In [7]:
import cv2
import numpy as np

# Load pre-trained object detection model
net = cv2.dnn.readNet("Assets/Trained_Model/yolov3/yolov3.weights", "Assets/Trained_Model/yolov3/yolov3.cfg")
layer_names = net.getLayerNames()
# Kiểm tra nếu net.getUnconnectedOutLayers() trả về danh sách giá trị vô hướng
try:
    output_layers = [layer_names[i - 1] for i in net.getUnconnectedOutLayers().flatten()]
except TypeError:
    output_layers = [layer_names[i - 1] for i in net.getUnconnectedOutLayers()]

# Load video
cap = cv2.VideoCapture("Assets/No_Violation_Video/No Violation 1 - Made with Clipchamp.mp4")

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    height, width, channels = frame.shape
    blob = cv2.dnn.blobFromImage(frame, 0.00392, (416, 416), (0, 0, 0), True, crop=False)
    net.setInput(blob)
    outs = net.forward(output_layers)

    for out in outs:
        for detection in out:
            scores = detection[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]
            if confidence > 0.5:
                # Object detected
                center_x = int(detection[0] * width)
                center_y = int(detection[1] * height)
                w = int(detection[2] * width)
                h = int(detection[3] * height)

                # Rectangle coordinates
                x = int(center_x - w / 2)
                y = int(center_y - h / 2)
                
                cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
                cv2.putText(frame, str(class_id), (x, y + 30), cv2.FONT_HERSHEY_PLAIN, 3, (0, 255, 0), 3)

    cv2.imshow("Frame", frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


KeyboardInterrupt: 

: 